# Reto 3

In [132]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [133]:
data_wine = pd.read_csv('./data/winequality-white.csv',sep=';')

In [134]:
x= data_wine.iloc[:,0:11]
y= data_wine.iloc[:,11:12]

In [135]:
xTrain, xTest, yTrain, yTest = train_test_split(x, y, test_size = 2000, random_state = 0)

In [136]:
newData= pd.concat([xTrain,yTrain], axis = 1)

In [137]:
dataTrain1= newData.sample(100)
xTrain1= dataTrain1.iloc[:,0:11]
yTrain1= dataTrain1.iloc[:,11]
dataTrain2= newData.sample(1000)
xTrain2= dataTrain2.iloc[:,0:11]
yTrain2= dataTrain2.iloc[:,11]
dataTrain3= newData.sample(2898)
xTrain3= dataTrain3.iloc[:,0:11]
yTrain3= dataTrain3.iloc[:,11]

In [149]:
def gradient_desc(nX,nY,minError=0.001):
    a=nX.to_numpy()
    y=nY.to_numpy()
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    H=[[0] * wNumber]* wNumber
    for i in range(0, numData):
        xi= x[i]
        H+=xi.transpose()*xi
    ei=np.linalg.eigvals(H)
    n=2/max(ei)
    ws= [0.01] * wNumber
    errorP=1
    error=0
    while abs(errorP-error) > minError:
        errorP=error
        error=0
        gradient=[0] * wNumber
        for i in range(0, numData):
            xs=(x[i]*ws).sum()
            xs=xs-y[i]
            gradient+=xs*x[i]
            error+=xs**2
        error=error/2
        gradient= gradient
        if error>errorP:
            n=n/2
            
        ws= ws - n*gradient
    return ws
    

In [148]:
linearRegr1 = gradient_desc(xTrain1, yTrain1)
print(linearRegr1)

[[ 6.    0.17  0.21 ...  0.54  9.8   1.  ]
 [ 6.9   0.24  0.49 ...  0.57 10.7   1.  ]
 [ 6.5   0.14  0.32 ...  0.74 11.5   1.  ]
 ...
 [ 7.3   0.19  0.27 ...  0.41  8.8   1.  ]
 [ 6.4   0.35  0.35 ...  0.5   9.8   1.  ]
 [ 5.7   0.33  0.15 ...  0.62  9.9   1.  ]]
[ 0.17875246+0.j  0.01389341+0.j  0.01735624+0.j  0.0269592 +0.j
  0.01100226+0.j  0.01426663+0.j -0.00048336+0.j  0.03661903+0.j
  0.09446654+0.j  0.02336905+0.j  0.35894674+0.j  0.03693765+0.j]


In [150]:
linearRegr2 = gradient_desc(xTrain2, yTrain2)
print(linearRegr2)

[ 0.16974321  0.01334639  0.0182935   0.01742394  0.01047753  0.01056167
 -0.00170159  0.03847483  0.10553474  0.02631504  0.38574942  0.03883492]


In [151]:
linearRegr3 = gradient_desc(xTrain3, yTrain3)
print(linearRegr3)

[ 0.13236914  0.00709865  0.01744015  0.02002596  0.01032451  0.01220242
 -0.00180592  0.04114162  0.12018848  0.0281921   0.39739599  0.04154561]


In [164]:
def calc_r2(nX,nY,w):
    a=nX.to_numpy()
    y=nY.to_numpy()
    x = np.ones((a.shape[0],a.shape[1]+1))
    x[:,:-1] = a
    wNumber= x.shape[1]
    numData= x.shape[0]
    u=0
    v=0
    ym= y.mean()
    err=0
    for i in range(0, numData):
        xi=x[i]
        yi=y[i]
        yp=(xi*w).sum()
        u+=(yi-yp)**2
        v+=(yi-ym)**2
    r2=1-(u/v)
    return r2
    

In [161]:
r21 = calc_r2(xTest, yTest,linearRegr1)
print(r21)

[1508.47611734+0.j]
[1632.238]
[0.07582343+0.j]


In [162]:
r22 = calc_r2(xTest, yTest,linearRegr2)
print(r22)

[1390.08798934]
[1632.238]
[0.1483546]


In [163]:
r23 = calc_r2(xTest, yTest,linearRegr3)
print(r23)

[1378.2280108]
[1632.238]
[0.15562068]
